In [ ]:
import pandas as pd

def load_backtest_data(file_path):
    """Loads backtest data and prepares it for model training."""
    df = pd.read_csv(file_path)
    df.dropna(inplace=True)  # Remove missing values
    return df

df = load_backtest_data("backtest_results.csv")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import joblib

def train_random_forest(df):
    """Trains a Random Forest model to classify profitable vs. losing trades."""
    X = df.drop(columns=["trade_outcome"])  # Features
    y = df["trade_outcome"]  # Target

    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X, y)

    joblib.dump(rf, "random_forest_model.pkl")  # Save model
    return rf

rf_model = train_random_forest(df)


In [ ]:
importances = rf_model.feature_importances_
feature_importance_df = pd.DataFrame({"feature": X.columns, "importance": importances}).sort_values(by="importance", ascending=False)
feature_importance_df.to_csv("feature_importance.csv", index=False)

In [ ]:
def filter_trade(trade_features):
    """Uses trained model to filter out high-risk trades."""
    rf_model = joblib.load("random_forest_model.pkl")
    prediction = rf_model.predict([trade_features])
    return prediction[0]  # 1 = Good Trade, 0 = High-Risk Trade